# Demo
1. Show two json files as the inputs
2. Post to terrarium funman api, show response id
3. Get result from id, and plot

# Todo
1. How to interact with terrarium
2. Setup API for two json files
3. Setup async anytime results
4. Truncate/halt request endpoint
5. Allow multiple requests in a queue, needs server thread

# Demo Setup

In [ ]:
# Common imports
import asyncio
import nest_asyncio
import os
import json
from pathlib import Path

from IPython.display import JSON

# Setup URLs
API_BASE_PATH = os.path.join(os.getcwd(), "..")
# Currently assumes a localhost server us running (see funman_server.ipynb)
API_SERVER_HOST = "127.0.0.1"
API_SERVER_PORT = 8190
SERVER_URL = f"http://{API_SERVER_HOST}:{API_SERVER_PORT}"
OPENAPI_URL = f"{SERVER_URL}/openapi.json"
CLIENT_NAME = "funman-api-client"

# Setup Paths
RESOURCES = Path("../resources").resolve()
# example paths
EXAMPLE_DIR = RESOURCES / "funman" / "infected_threshold_example"
MODEL_PATH = EXAMPLE_DIR / "model.json"
REQUEST_PATH = EXAMPLE_DIR / "request.json"

In [ ]:
# Alternative example
EXAMPLE_DIR = RESOURCES / "common_model" / "petrinet"
MODEL_PATH = EXAMPLE_DIR / "sir.json"
REQUEST_PATH = EXAMPLE_DIR / "request.json"

In [ ]:
# Install API client to the working directory
# and add the install to the sys path
import funman.api.client as client
client.make_client(
    API_BASE_PATH, openapi_url=OPENAPI_URL, client_name=CLIENT_NAME
)
nest_asyncio.apply()

# Import and initialize the FUNMAN API Client

In [ ]:
# Setup API client request
from funman_api_client import Client
from funman_api_client.api.default import (
    post_queries_queries_post,
    get_queries_queries_query_id_get,
)
from funman_api_client.models.body_post_queries_queries_post import BodyPostQueriesQueriesPost
from funman_api_client.models import (
    BilayerModel,
    FUNMANConfig,
    LabeledParameter,
    QueryLE,
    FunmanWorkRequest,
    FunmanWorkUnit,
    FunmanResults,
    QueryTrue,
    GeneratedPetriNetModel
    
)

# Create a funman client
funman_client = Client(SERVER_URL, timeout=None)

# Read in the input files

In [ ]:
# Read in the model file
model = json.loads(MODEL_PATH.read_bytes())
# pretty print
print(json.dumps(model, indent=2))

In [ ]:
# Read in the funman request file
request = json.loads(REQUEST_PATH.read_bytes())
# pretty print
print(json.dumps(request, indent=2))

# Make a request

In [ ]:
# Make a POST request to the API endpoint
response = asyncio.run(
    post_queries_queries_post.asyncio_detailed(
        client=funman_client,
        json_body=BodyPostQueriesQueriesPost.from_dict({
            "model": model,
            "request": request
        }),
    )
)

# The response returns a work unit
work_unit = FunmanWorkUnit.from_dict(
    src_dict=json.loads(response.content.decode())
)
# Where the id can be used to pull ongoing results
work_unit.id

# Request results

In [ ]:
# Make a GET request to the API endpoint
response = asyncio.run(get_queries_queries_query_id_get.asyncio_detailed(
    work_unit.id,
    client=funman_client
))
# The response returns the latest funman results for the query
results = FunmanResults.from_dict(
    src_dict=json.loads(response.content.decode())
)
# pretty print the parameter_space
print(json.dumps(results.parameter_space.to_dict(), indent=2))

# Plot

In [ ]:
# Plot the ParameterSpace
from funman_demo.parameter_space_plotter import ParameterSpacePlotter
ParameterSpacePlotter(results.parameter_space).plot(show=True)